In [28]:
from pyspark.sql.functions import asc, first, mean, row_number, when, udf, stddev_pop, col, lit, collect_list, array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.ml.linalg import DenseVector

## Load Dataset
The dataset is loaded from MariaDB into a dataframe.

In [29]:
spark = SparkSession.builder.appName('ReadMariaDB') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.getOrCreate()


sql = "select * from 01_sampled_games_2v2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
database = "steam"
user = "root"
password = "example"
server = "192.168.2.62"
port = 3306
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"
jdbc_driver = "org.mariadb.jdbc.Driver"

# Create a data frame by reading data from Oracle via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

df = df.drop("playtime_2weeks", "dateretrieved")

In [33]:
# Count the number of rows in the DataFrame
row_count = df.count()

# Print the row count
print("The DataFrame has", row_count, "rows.")

df.show()

The DataFrame has 88349 rows.
+-----------------+-----+----------------+
|          steamid|appid|playtime_forever|
+-----------------+-----+----------------+
|76561197960268000|  300|             109|
|76561197960268000| 1300|              94|
|76561197960268000| 2100|             110|
|76561197960268000| 4000|             152|
|76561197960268000| 2600|              59|
|76561197960268000| 9000|               2|
|76561197960268000| 2300|              40|
|76561197960268000| 2200|             210|
|76561197960268000| 4500|            1002|
|76561197960268000|  400|             380|
|76561197960268000|17300|            4312|
|76561197960268000|  500|             198|
|76561197960268000|22300|             425|
|76561197960268000|19900|             111|
|76561197960268000|18500|            1160|
|76561197960268000|22200|              20|
|76561197960268000|35700|             433|
|76561197960268000|36000|              27|
|76561197960268000|39800|             510|
|76561197960268000|32400

## Pearson Correlation Matrix
Since the dataset contains at most ~4500 games, we can expect a 4500^2=81,000,000 sized matrix. Each float entry takes 4 bytes in memory. Therefore, the pearson correlation matrix would take up 324 MB of memory.

Since the memory used is relatively small, we will pre-compute the person correlation matrix and store it for use later in the algorithm.

In [32]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, DoubleType
# Create playtime vectors for each game

list_to_dense = udf(lambda l: Vectors.dense([v for (k,v) in sorted(l)]), VectorUDT())
vectors = df.groupBy('appid').agg(collect_list('playtime_forever'))\
                                  .withColumn('features', list_to_dense('collect_list(playtime_forever)'))\
                                  .select('features')
vectors.collect()

Py4JJavaError: An error occurred while calling o666.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 56.0 failed 1 times, most recent failure: Lost task 0.0 in stage 56.0 (TID 37) (DESKTOP-Q5EKQ2B.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.R

In [ ]:
pearson_matrix = Correlation.corr(vectors, "features", "pearson")
corr_array = pearson_matrix.head()[0].toArray()
print(corr_array)

## Per-Game 1-5 Rating Normalization
For each game, we calculate the mean and standard deviation. We then create buckets for each rating:
### Cut points
* Cut point 1: mean - std_dev*0.5 if > 0, else 0
* Cut point 2: mean
* Cut point 3: mean + std_dev*0.5
* Cut point 4: mean + std_dev
### Ratings
* Rating 1: 0 < x < cut point 1
* Rating 2: cut point 1 < x < cut point 2
* Rating 3: cut point 2 < x < cut point 3
* Rating 4: cut point 3 < x < cut point 4
* Rating 5: cut point 5 < x < inf

In [ ]:

# Calculate the per-game mean and standard deviation of the playtime column
game_stats = df.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)
df = df.join(game_stats, "appid")

df = df.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5)).otherwise(0))
df = df.withColumn("cut_point_2", col("game_mean_playtime"))
df = df.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5))
df = df.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime"))

df = df.withColumn(
    "ratings",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

df.show()